In [118]:
def get_data(fname):
    with open(fname) as f:
        my_hexdata = f.readlines()[0]
    scale = 16 ## equals to hexadecimal
    num_of_bits = len(my_hexdata)*4
    return bin(int(my_hexdata, scale))[2:].zfill(num_of_bits)

In [197]:
def get_literal(bin_data):
    res = ''
    for i in range(0, len(bin_data), 5):
        if bin_data[i] == '1':
            res += bin_data[i+1:i+5]
        else:
            res += bin_data[i+1:i+5]
            return (res, i+5)

In [297]:
def get_packet(data):
    version = int(data[0:3], 2)
    type_id = int(data[3:6], 2)
    data = data[6:]
    i = 6
    
    if type_id == 4:
        numb, ind = get_literal(data)
        return (type_id, 'literal', numb, i+ind)
    else:
        len_type_id = data[0]
    
        data = data[1:]
        i+=1
    
        if len_type_id == '0':
            sp_len = int(data[0:15], 2)
            return (type_id, 'len', sp_len, i+15)
        else:
            no_sp = int(data[0:11], 2)
            return (type_id, 'num', no_sp, i+11)

In [332]:
def do_stuff(binary_data):
    version, p_type, content, index = get_packet(binary_data)
    print(get_packet(binary_data))
    binary_data = binary_data[index:]
    tot_vers = version
    if p_type == 'len':
        cur_len = 0
        w_ind = 0
        while w_ind < content:
            temp = len(binary_data)
            vers, binary_data = do_stuff(binary_data)
            tot_vers += vers
            w_ind += temp - len(binary_data)
#             print(f'wind {w_ind} / {content}')
    elif p_type == 'num':
        for i in range(content):
            vers, binary_data = do_stuff(binary_data)
            tot_vers += vers
    return tot_vers, binary_data

In [380]:
def do_stuff(binary_data): #remade for part 2
    version, p_type, content, index = get_packet(binary_data)
    eqn = [get_packet(binary_data)]
    binary_data = binary_data[index:]
    if p_type == 'len':
        cur_len = 0
        w_ind = 0
        while w_ind < content:
            temp = len(binary_data)
            eq, binary_data = do_stuff(binary_data)
            eqn.append(eq)
            w_ind += temp - len(binary_data)
#             print(f'wind {w_ind} / {content}')
    elif p_type == 'num':
        for i in range(content):
            eq, binary_data = do_stuff(binary_data)
            eqn.append(eq)
    return eqn, binary_data

In [374]:
def get_fvalue(eqn):
    if eqn[0][0] == 4:
        return int(eqn[0][2],2)
    
    values = [get_fvalue(sub) for sub in eqn[1:]]
    
    if eqn[0][0] == 0:
        return sum(values)
    elif eqn[0][0] == 1:
        prod = 1
        for value in values:
            prod *= value
        return prod
    elif eqn[0][0] == 2:
        return min(values)
    elif eqn[0][0] == 3:
        return max(values) 
    elif eqn[0][0] == 5:
        return 1 if values[0] > values[1] else 0
    elif eqn[0][0] == 6: # less than
        return 1 if values[0] < values[1] else 0
    else:
        return 1 if values[0] == values[1] else 0

In [379]:
binary_data = get_data('input.txt')
equation = do_stuff(binary_data)[0]
print(get_fvalue(equation))

7936430475134
